In [27]:
import pandas as pd
import numpy as np
import time

from bs4 import BeautifulSoup
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException


def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [29]:
# 수집할 맥주 목록
beer_list = ['kloud', 'fitz super clear', 'Asahi super dry', 'Tsingtao', 'Heineken',
 'Kirin ichiban', 'Sapporo Premium Beer / Draft Beer', 'stella artois', 'guinness braught',
 '1664 Blanc', 'pilsner urquell', 'San Miguel', 'OB premier pilsner', 'cass fresh',
 'Steam Brew Imperial Stout', 'dry finish', 'max hite', 'hite extra cold', 'victoria bitter', 'BINTANG pilsner',
 'krombacher weizen', 'Miller Genuine Draft', 'Hoegaarden Cherry', 'TIGER REDLER',
 "Suntory The Premium Malt's", 'REEPER B Weissbier', 'PEEPER B IPA', 'TIGER BEER',
 'TSINGTAO WHEAT BEER', 'Erdinger Weissbier', 'Carlsberg', 'Budweiser ', 'Hoegarden',
 'YEBISU', 'Paulaner Hefe-weissbier', 'Desperados', 'Peroni Nastro Azzurro',
 'Edelweiss Snowfresh', 'Heineken Dark Lager', 'Kozel Dark 10', 'Guinness original',
 'Filite','Jeju Wit Ale', 'Stephans Brau Philsner', 'Stephans Brau Larger',
 'Stephans Bräu Hefe-Weizen Naturtrüb', 'Bali Hai Premium Larger', 'Apostel Brau',
 'Egger Zwickl', 'Egger Marzenbier', 'Holsten Premium Beer', 'Franzisaner Hefe-Weissbier',
 'Egger Radler Grapefruit', 'Barvaria Premium', 'Barvaria 8.6', 'Lapin Kulta IV A',
 'Grolsch Premium Larger', 'Gambrinus Original', 'XXXX Gold', 'Leffe Brown',
 'Lowenbrau Original', 'Asahi Super dray Black ', 'Harbin Beer', "beck's",
 'Hoegaarden Rosee', 'Platinum White Ale', 'Platinum Pale Ale', 'Ambar Especial Larger',
 'Schöfferhofer Grapefruit', 'Volfas Engelman Grünberger Hefeweizen', 'Berliner Kindl Pilsener ',
 'BURGE MEESTER', 'Red Rock', 'Erdinger Dunkel', 'Warsteiner Premium Verum', "Queen's Ale Blonde Type"]

# 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

In [5]:
beer_list

,검색이름
0,kloud
1,fitz super clear
2,Asahi super dry
3,Tsingtao
4,Heineken
...,...
71,BURGE MEESTER
72,Red Rock
73,Erdinger Dunkel
74,Warsteiner Premium Verum


In [30]:
# 데이터 프레임 생성
data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

# chromedriver.exe 파일 경로 설정
chromedriver = 'chromedriver.exe'
# 크롤링 할 경로 설정
url = 'https://www.ratebeer.com/search?tab=beer'


browser = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
browser.get(url)
browser.set_window_size(900, 900)
time.sleep(1)

In [55]:
def crawl(browser, beer, data, k):
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url='https://www.ratebeer.com/'
    browser.get(url)
    browser.set_window_size(900, 900)
    #  맥주 검색
    time.sleep(2)
    element = browser.find_element(By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div[2]/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)
    # 상품 선택
    browser.find_element(By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div[2]/div[2]/a[1]/div/div[2]').click()

    # 상품 이름 수집
    time.sleep(3)
    beer_name = browser.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text

    error_cnt = 0

    while 1:
        try :
            # 전체 리뷰 개수 수집
            time.sleep(3)
            string = browser.find_element(By.CLASS_NAME,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text

            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # Score breakdown 클릭
    time.sleep(3)
    element = browser.find_element(By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    time.sleep(3)
    # 해당 element로 이동하는 코드
    browser.execute_script("arguments[0].click();", element)

    # 수집할 Page 수를 계산
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')

        # 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        html = browser.page_source
        soup=BeautifulSoup(html,'html.parser')
        beer_info=soup.select('.px-4.fj-s.f-wrap')
        tmp = []
        for info in beer_info:
            text = info.text
            tmp.append(text)
        # 수집한 것을 데이터프레임에 저장
        # "더 보기" 버튼이 있는지 확인하고, 있으면 클릭하여 추가 리뷰를 로드합니다.
        
        show_more_button = None
        try:
            time.sleep(2)
            show_more_button = info.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[3]/div[4]/div/section[2]/div[1]/div/button/span[1]')
        except NoSuchElementException:
            pass

        if show_more_button is not None and show_more_button.is_displayed():
            browser.execute_script("arguments[0].click();", show_more_button)
            time.sleep(2)



        # 다음 페이지로 넘어가기 
        try :
            element = browser.find_element(By.XPATH,'//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(3)
            browser.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    # 데이터가 중복 수집될 경우 리뷰 수 만큼만 Cut
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))

    # 데이터를 csv, excel 파일로 저장
    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')

    browser.quit()

    return result


In [56]:
# 맥주 리스트 개수만큼 데이터 수집
for k in range(len(beer_list)):
    result = crawl(browser, beer_list['검색이름'].iloc[k], data, k)

url_open... kloud 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
1 번째 페이지입니다.


TypeError: 'NoneType' object is not callable

In [48]:
import pandas as pd

# 합친 데이터를 저장할 데이터프레임
data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

# 수집한 파일의 개수
files_cnt = 76

for i in range(files_cnt):

    # 해당 경로에서 beer_n_1.csv 형식의 파일들만 수집한 뒤 병합
    try : 
        tmp = pd.read_csv(r'C:\Users\sun14\Desktop\졸작\beer_n_'+str(i)+'.csv', index_col=0)
        data = pd.concat([data,tmp])
    # 오류 발생 시 넘어감
    except :
        print(i, '번째에서 오류가 발생했습니다. 다음 파일로 넘어갑니다.')

# 합친 데이터를 저장합니다.
data.to_csv('전처리전데이터.csv', encoding='utf-8')

In [16]:
df=pd.read_csv('전처리전데이터.csv')

In [5]:
df=pd.read_csv('beer2_n_2.csv')

In [6]:
df

,Unnamed: 0,맥주정보,검색이름,맥주이름
0,0,Rated: 2.79 by lakewy17 from New HampshireApr ...,Heineken,Heineken Lager Beer
1,1,Reviewed by dano213 from Pennsylvania3.33/5 r...,Heineken,Heineken Lager Beer
2,2,Rated by Davidlbarnes from Georgia5/5 rDev +8...,Heineken,Heineken Lager Beer
3,3,"Rated: 3.67 by morsa_95 from CaliforniaMar 21,...",Heineken,Heineken Lager Beer
4,4,Reviewed by dbl_delta from Pennsylvania2.3/5 ...,Heineken,Heineken Lager Beer
...,...,...,...,...
5671,5671,Rated by John from New York2.26/5 rDev -17.5%...,Heineken,Heineken Lager Beer
5672,5672,Rated by geckointhebox1/5 rDev -63.5%look: 1 ...,Heineken,Heineken Lager Beer
5673,5673,Rated by xXTequila from Massachusetts1.48/5 r...,Heineken,Heineken Lager Beer
5674,5674,Rated by grdahl from Florida2/5 rDev -27%look...,Heineken,Heineken Lager Beer


In [ ]:
df = pd.read_csv('전처리전데이터.csv', encoding='utf-8', index_col=0)

df = df[['맥주이름', '맥주정보']]

df

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5002 entries, 0 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   맥주이름    5002 non-null   object
 1   맥주정보    5002 non-null   object
dtypes: object(2)
memory usage: 117.2+ KB


FileNotFoundError: [Errno 2] No such file or directory: '크롤링_원본데이터.csv'